In [1]:
import sys
from pathlib import Path
import pandas as pd

ruta_raiz = Path("..").resolve()
if str(ruta_raiz) not in sys.path:
    sys.path.append(str(ruta_raiz))

from src.analitica_spotify.consultas import (
    top_artistas,
    minutos_por_hora,
    minutos_por_dia_semana,
    minutos_por_anio_mes,
    indice_obsesion,
    resumen_entre_semana_vs_fin,
    minutos_por_bloque_horario,
    dia_mas_musical,
    racha_musical_mas_larga,
    resumen_variabilidad_diaria,
    top_canciones,
    artistas_emergentes_y_olvidados,
)

In [2]:
ruta_elias = Path("../datos/procesados/elias_limpio.csv")
ruta_elie = Path("../datos/procesados/elie_limpio.csv")

df_elias = pd.read_csv(ruta_elias, parse_dates=["fecha_reproduccion"])
df_elie = pd.read_csv(ruta_elie, parse_dates=["fecha_reproduccion"])

df_elias["usuario"].unique(), df_elie["usuario"].unique()

(array(['Elias'], dtype=object), array(['elie'], dtype=object))

In [3]:
df_conjunto = pd.concat([df_elias, df_elie], ignore_index=True)
df_conjunto["usuario"].value_counts()

usuario
Elias    23401
elie     17880
Name: count, dtype: int64

In [4]:
df_elias2 = df_conjunto[df_conjunto["usuario"] == "Elias"].copy()
df_elie2 = df_conjunto[df_conjunto["usuario"] == "elie"].copy()
#Parece redundante porque ya tenemos df_elias y df_elie, pero es importante para acostumbrarnos a la logica de "filtrar por usuario" para el Dashboard.

In [6]:
top_artistas(df_elie2, n=10)

artista
Bad Bunny           2333.956433
Billie Eilish       1574.885700
Drake               1284.598683
The Weeknd          1163.198333
Ludovico Einaudi    1031.841367
Jesse & Joy         1016.144700
Hans Zimmer         1014.437217
Frank Ocean          901.755600
Adele                824.820117
Fairuz               761.744533
Name: minutos_reproducidos, dtype: float64

In [11]:
obs1_elias = indice_obsesion(df_elias2, n=1)
obs1_elie = indice_obsesion(df_elie2, n=1)
obs5_elias = indice_obsesion(df_elias2, n=5)
obs5_elie = indice_obsesion(df_elie2, n=5)
obs10_elias = indice_obsesion(df_elias2)
obs10_elie = indice_obsesion(df_elie2)

pd.DataFrame({
    "usuario": ["Elias", "elie"],
    "indice_obsesion_top1": [obs1_elias, obs1_elie],
    "indice_obsesion_top5": [obs5_elias, obs5_elie],
    "indice_obsesion_top10": [obs10_elias, obs10_elie]
})

,usuario,indice_obsesion_top1,indice_obsesion_top5,indice_obsesion_top10
0,Elias,12.12,25.88,33.38
1,elie,5.01,15.85,25.54


In [18]:
mins_elias = minutos_por_anio_mes(df_elias2).assign(usuario="Elias")
mins_elie = minutos_por_anio_mes(df_elie2).assign(usuario="elie")
mins_conjunto = pd.concat([mins_elias, mins_elie], ignore_index=True)
mins_conjunto.head()


,anio,mes,minutos_reproducidos,usuario
0,2024,11,330.839867,Elias
1,2024,12,2839.850033,Elias
2,2025,1,3102.822450,Elias
3,2025,2,3455.191083,Elias
4,2025,3,3449.969550,Elias


In [19]:
semana_elias = resumen_entre_semana_vs_fin(df_elias2)
semana_elias["usuario"] = "Elias"
semana_elie = resumen_entre_semana_vs_fin(df_elie2)
semana_elie["usuario"] = "elie"

semana_conjunto = pd.concat([semana_elias, semana_elie], ignore_index=True)
semana_conjunto

,grupo,minutos_totales,minutos_promedio_por_dia,dias_con_musica,usuario
0,entre_semana,24990.297500,121.312124,206,Elias
1,fin_de_semana,11434.744400,84.079003,136,Elias
2,entre_semana,32496.224983,156.231851,208,elie
3,fin_de_semana,14121.195967,98.749622,143,elie


In [20]:
bloques_elias = minutos_por_bloque_horario(df_elias2).assign(usuario="Elias")
bloques_elie = minutos_por_bloque_horario(df_elie2).assign(usuario="elie")
bloques_conjunto = pd.concat([bloques_elias, bloques_elie], ignore_index=True)
bloques_conjunto

,bloque_horario,minutos_reproducidos,usuario
0,madrugada,8555.082933,Elias
1,manana,500.452850,Elias
2,tarde,11435.529833,Elias
3,noche,15933.976283,Elias
4,madrugada,11636.764300,elie
5,manana,1776.403417,elie
6,tarde,17276.461317,elie
7,noche,15927.791917,elie


In [25]:
niveles = [5, 10, 30, 60, 120]

streaks_elias = {
    nivel: racha_musical_mas_larga(df_elias2, umbral_minutos_dia=nivel)
    for nivel in niveles
}

streaks_elie = {
    nivel: racha_musical_mas_larga(df_elie2, umbral_minutos_dia=nivel)
    for nivel in niveles
}

streaks_elias, streaks_elie


({5: {'longitud_racha': 32,
   'fecha_inicio': datetime.date(2025, 10, 13),
   'fecha_fin': datetime.date(2025, 11, 13)},
  10: {'longitud_racha': 32,
   'fecha_inicio': datetime.date(2025, 10, 13),
   'fecha_fin': datetime.date(2025, 11, 13)},
  30: {'longitud_racha': 19,
   'fecha_inicio': datetime.date(2024, 12, 2),
   'fecha_fin': datetime.date(2024, 12, 20)},
  60: {'longitud_racha': 10,
   'fecha_inicio': datetime.date(2025, 1, 22),
   'fecha_fin': datetime.date(2025, 1, 31)},
  120: {'longitud_racha': 8,
   'fecha_inicio': datetime.date(2025, 11, 20),
   'fecha_fin': datetime.date(2025, 11, 27)}},
 {5: {'longitud_racha': 118,
   'fecha_inicio': datetime.date(2025, 7, 21),
   'fecha_fin': datetime.date(2025, 11, 15)},
  10: {'longitud_racha': 90,
   'fecha_inicio': datetime.date(2025, 8, 18),
   'fecha_fin': datetime.date(2025, 11, 15)},
  30: {'longitud_racha': 26,
   'fecha_inicio': datetime.date(2025, 8, 18),
   'fecha_fin': datetime.date(2025, 9, 12)},
  60: {'longitud_racha'

In [27]:
var_elias = resumen_variabilidad_diaria(df_elias2)
var_elie = resumen_variabilidad_diaria(df_elie2)

var_elias, var_elie

({'promedio_minutos_por_dia': 106.51,
  'desviacion_estandar': 69.06,
  'percentil_25': 53.31,
  'percentil_50': 101.9,
  'percentil_75': 151.06,
  'dias_altos': 74,
  'dias_bajos': 86,
  'total_dias': 342},
 {'promedio_minutos_por_dia': 132.81,
  'desviacion_estandar': 96.43,
  'percentil_25': 63.75,
  'percentil_50': 117.59,
  'percentil_75': 184.22,
  'dias_altos': 74,
  'dias_bajos': 95,
  'total_dias': 351})

In [28]:
dia_top_elias = dia_mas_musical(df_elias2)
dia_top_elie = dia_mas_musical(df_elie2)

dia_top_elias, dia_top_elie

(        fecha  minutos_reproducidos     top_artista  minutos_top_artista
 0  2025-04-09            332.882383  Eladio Carrion             65.89775,
         fecha  minutos_reproducidos top_artista  minutos_top_artista
 0  2025-03-13              593.9978   Bad Bunny            43.673333)

In [29]:
top_canciones_elias = top_canciones(df_elias2, n=10)
top_canciones_elie = top_canciones(df_elie2, n=10)

top_canciones_elias, top_canciones_elie

(                                                cancion       artista  \
 225                                   BAILE INoLVIDABLE     Bad Bunny   
 1283                                          KLOuFRENS     Bad Bunny   
 1955  Pangea (feat. Nathan Nicholson) - Maxi Meraki ...  Mahmut Orhan   
 2754                                     VUELVE CANDY B     Bad Bunny   
 2396                              Spacer - Extended Mix       Mooglie   
 2953                                               Yumi    Notre Dame   
 1850                                  Nothing Around Us       Mathame   
 1751                                         NADIE SABE     Bad Bunny   
 2802                                            WELTiTA     Bad Bunny   
 1281                                         KETU TeCRÉ     Bad Bunny   
 
       minutos_reproducidos  
 225             280.056917  
 1283            234.467633  
 1955            203.090050  
 2754            190.260117  
 2396            177.886433  
 295

In [30]:
top_canciones_elias["usuario"] = "Elias"
top_canciones_elie["usuario"]  = "elie"

top_canciones_conjunto = pd.concat([top_canciones_elias, top_canciones_elie], ignore_index=True)
top_canciones_conjunto

,cancion,artista,minutos_reproducidos,usuario
0,BAILE INoLVIDABLE,Bad Bunny,280.056917,Elias
1,KLOuFRENS,Bad Bunny,234.467633,Elias
2,Pangea (feat. Nathan Nicholson) - Maxi Meraki ...,Mahmut Orhan,203.090050,Elias
3,VUELVE CANDY B,Bad Bunny,190.260117,Elias
4,Spacer - Extended Mix,Mooglie,177.886433,Elias
5,Yumi,Notre Dame,176.819317,Elias
6,Nothing Around Us,Mathame,151.303017,Elias
7,NADIE SABE,Bad Bunny,148.751367,Elias
8,WELTiTA,Bad Bunny,148.163367,Elias
9,KETU TeCRÉ,Bad Bunny,147.257733,Elias


In [31]:
var_artistas_elias = artistas_emergentes_y_olvidados(df_elias2, top_n = 5)
var_artistas_elie = artistas_emergentes_y_olvidados(df_elie2, top_n = 5)

var_artistas_elias, var_artistas_elie

({'emergentes': mitad      artista          H1           H2        delta
  0      Luis Miguel  166.470350  1256.623267  1090.152917
  1             Mora  443.453417   767.470333   324.016917
  2        José José    0.000000   284.113167   284.113167
  3      Myke Towers  202.443533   400.043233   197.599700
  4        Omer Adam   30.221900   212.490400   182.268500,
  'olvidados': mitad        artista           H1           H2       delta
  0          Bad Bunny  2644.839533  1770.948067 -873.891467
  1         The Weeknd   611.008033   214.919083 -396.088950
  2              Drake   316.372750   109.364033 -207.008717
  3      Lewis Capaldi   364.100950   180.173850 -183.927100
  4       Travis Scott   292.840850   130.595050 -162.245800},
 {'emergentes': mitad       artista         H1          H2       delta
  0       Jesse & Joy  80.913633  935.231067  854.317433
  1      Aleks Syntek   0.023217  369.141683  369.118467
  2           berlioz   0.000000  229.163050  229.163050
  3     

In [13]:
from pathlib import Path
import pandas as pd

def buscar_raiz(nombre_proyecto="Proyecto_Spotify_Analytics"):
    p = Path().resolve()
    while p.name != nombre_proyecto and p != p.parent:
        p = p.parent
    return p

RUTA_RAIZ = buscar_raiz()
ruta = RUTA_RAIZ / "datos" / "aux" / "imagenes_artistas.csv"

print("Ruta detectada:", ruta)
print("Existe?:", ruta.exists())

df_img = pd.read_csv(ruta)
print(df_img.columns)
print(df_img["usuario"].unique())
df_img.head()



Ruta detectada: /Users/eliaskablym/Desktop/Proyecto_Spotify_Analytics/datos/aux/imagenes_artistas.csv
Existe?: True
Index(['usuario', 'artista', 'url_imagen'], dtype='object')
['Elias' 'elie']


,usuario,artista,url_imagen
0,Elias,Bad Bunny,assets/artists/badbunny.jpg
1,Elias,Eladio Carrion,assets/artists/eladio.jpg
2,Elias,Luis Miguel,assets/artists/luismiguel.jpg
3,Elias,Mora,assets/artists/mora.jpg
4,Elias,The Weeknd,assets/artists/theweeknd.jpg
